# Step 4: PostProcess Results

In this part, we post process the optimized results for evaluation and visualization.
At first, we prepare the following parameters:

| Parameter | Description | Value |
|--- |--- |--- |
|method | annealing method for QUBO problem |'dwave-sa': use the simulated annealer in ocean toolkit<br> 'dwave-qa': use the quantum annealer|
|raw_path| the path for the original molecule file| './molecule-data/117_ideal.mol2' in this example |
|data_path| the path for the processed molecule file| './qmu_117_ideal_data_latest.mol2' in this example |
|bucket | the s3 bucket to store your results | - |
|prefix | the name of the folder in your s3 bucket | - |
|task_id | the id for your quantum annealing task| please check the value in the previous step|

Then we can run the post-process using **ResultParser** object for SA:

In [ ]:
from utility.MoleculeParser import MoleculeData
from utility.QMUQUBO import QMUQUBO
from utility.AnnealerOptimizer import Annealer
from utility.ResultProcess import ResultParser
import time

timestamp = time.strftime("%Y%m%d-%H")

In [27]:
method = "dwave-sa"
sa_param = {}
sa_param["raw_path"] = raw_path
sa_param["data_path"] = data_path

sa_process_result = ResultParser(method, **sa_param)
# print(f"{method} result is {sa_process_result.get_all_result()}")

local_time, _ , _, _= sa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time}")

INFO:root:_load_raw_result
INFO:root:load simulated annealer raw result
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse simulated annealer result
INFO:root:sa only has local_time!


time for dwave-sa: 
     local time is 129.5549328327179


In [28]:
sa_atom_pos_data = sa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
sa_result_filepath, sa_result_json = sa_process_result.save_mol_file(f"{timestamp}")

print(f"result path is {sa_result_filepath}, and result optimization file path is {sa_result_json}")

INFO:root:generate_optimize_pts()
INFO:root:var_dict_raw {'14': ['1'], '15': ['8']} var_dict_list [{'14': '1', '15': '8'}]
INFO:root:chosen var {'x_14_1', 'x_15_8'}
INFO:root:tor list {'X_15_8', 'X_14_1'}
INFO:root:initial 18.549876807564765
INFO:root:optimize 18.56185411730613
INFO:root:optimize_gain 1.0006456813630418
INFO:root:start physical check
INFO:root:save_mol_file 20220323-14
INFO:root:finish save ./molecule-data/117_ideal_dwave-sa_20220323-14.mol2 and ./molecule-data/117_ideal_dwave-sa_20220323-14.json


result path is ./molecule-data/117_ideal_dwave-sa_20220323-14.mol2, and result optimization file path is ./molecule-data/117_ideal_dwave-sa_20220323-14.json


In [29]:
sa_process_result.parameters

{'volume': {'optimize': 18.56185411730613,
  'initial': 18.549876807564765,
  'gain': 1.0006456813630418,
  'unfolding_results': ['X_15_8', 'X_14_1'],
  'annealing_results': ['x_14_1', 'x_15_8'],
  'optimize_info': {'optimize_state': True, 'result_rank': 11}}}

In the first block, we can see the **local time**
for SA is more than 150 seconds. 
With the **generate_optimize_pts()** method, the final 3D 
points after unfolding will be generated and saved as json file and mol2 files. The last 
block shows the optimizing results which are also stored in json files. 
It shows that the optimized result gains 
1.0006x increase in volume. The value for **unfolding_results** indicates 
that the rotatable bond 15 should rotate $270^o$ ($360/8*(7-1)$) and 
the rotatable bond 14 should rotate $0^o$ ($360/8*(1-1)$).
At the same time, you can run the post-process for QA:

In [30]:
method = "dwave-qa"
qa_param = {}
qa_param["bucket"] = s3_bucket
qa_param["prefix"] = prefix
qa_param["task_id"] = qa_task_id
qa_param["raw_path"] = raw_path
qa_param["data_path"] = data_path

qa_process_result = ResultParser(method, **qa_param)
# print(f"{method} result is {qa_process_result.get_all_result()}")

local_time, task_time, total_time, access_time = qa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time},\n \
    task time is {task_time}, \n \
    qpu total time is {total_time}, \n \
    qpu access time is {access_time}")

INFO:root:_load_raw_result
INFO:root:load quantum annealer raw result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: annealer-experiment/22419429-2f93-4679-9305-a2e099dced7c/qa_result.pickle
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse quantum annealer result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: annealer-experiment/22419429-2f93-4679-9305-a2e099dced7c/results.json


time for dwave-qa: 
     local time is 11.894113779067993,
     task time is 3.327, 
     qpu total time is 0.144422, 
     qpu access time is 0.129189


we can see that there many types of time metrics for running QA.
This task has the **local time** of 5 s, which means the time between calling the api and 
getting the annealing result. The **task time** time is the metric from the json file in 
bucket. We can also see the **qpu total time** and **qpu access time** representing the 
actual time running in the QPU. Please refer to [Operation and Timing](https://docs.dwavesys.com/docs/latest/c_qpu_timing.html)
for details.

In [31]:
qa_atom_pos_data = qa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
qa_result_filepath, qa_result_json = qa_process_result.save_mol_file(f"{timestamp}")
print(f"result path is {qa_result_filepath}, and result optimization file path is {qa_result_json}")

INFO:root:generate_optimize_pts()
INFO:root:var_dict_raw {'14': ['1'], '15': ['8']} var_dict_list [{'14': '1', '15': '8'}]
INFO:root:chosen var {'x_14_1', 'x_15_8'}
INFO:root:tor list {'X_15_8', 'X_14_1'}
INFO:root:initial 18.549876807564765
INFO:root:optimize 18.56185411730613
INFO:root:optimize_gain 1.0006456813630418
INFO:root:start physical check
INFO:root:save_mol_file 20220323-14
INFO:root:finish save ./molecule-data/117_ideal_dwave-qa_20220323-14.mol2 and ./molecule-data/117_ideal_dwave-qa_20220323-14.json


result path is ./molecule-data/117_ideal_dwave-qa_20220323-14.mol2, and result optimization file path is ./molecule-data/117_ideal_dwave-qa_20220323-14.json


In [32]:
qa_process_result.parameters

{'volume': {'optimize': 18.56185411730613,
  'initial': 18.549876807564765,
  'gain': 1.0006456813630418,
  'unfolding_results': ['X_15_8', 'X_14_1'],
  'annealing_results': ['x_14_1', 'x_15_8'],
  'optimize_info': {'optimize_state': True, 'result_rank': 1}}}

In the same way, the optimized results are translated the 3D points and saved 
as local json and mol2 files. The result indicates that QA gains 
1.0006x increase in 
volume.

Finally, We can open folders for the optimized results:

![optimize-results](../../../images/optimize-results.png)

Figure 1: Optimized results

We can see the json result and mol2 file of SA and QA are 
stored in this place. If we carry out more 
experiments, more results with time stamp are 
stored incrementally. 
For visualization, we can use the following method to see the original the molecule and unfolding molecule.

In [37]:
# this shows the original molecule
qa_process_result.InteractView(raw_path, size=800)

interactive(children=(Text(value='./molecule-data/117_ideal.mol2', description='mol'), IntSlider(value=800, de…

In [38]:
# copy the variable for the path of unfolding molecule, we can see the unfolding results. this also works for sa_process_result.
qa_process_result.InteractView(qa_result_filepath, size=800)

interactive(children=(Text(value='./molecule-data/117_ideal_dwave-qa_20220323-14.mol2', description='mol'), In…

Alternatively, we can upload the mol2 file into 
[online viewer tool](https://www.rcsb.org/3d-view) 
to see the result:

![visual](../../../images/visualization.png)

Figure 2: Visualization